# Amazon Titan Embeddings を使用したベクトル化
埋め込みモデルの Amazon Titan Embeddings を使用して、テキストをベクトル化します。
ベクトル化されたテキストの距離を比較することで、テキストの類似度を計算することができます。

In [13]:
import json
import os
import sys

import boto3
import botocore

import numpy as np
from numpy.linalg import norm

module_path = ".."
sys.path.append(os.path.abspath(module_path))

AWS Python SDK の invoke_model を使用してベクトル化したいテキストを埋め込みモデルにインプットします。

In [14]:
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

# 埋め込みを行う基盤モデルを使用
modelId="amazon.titan-embed-text-v1"
# modelId="amazon.titan-embed-text-v2:0" ラボ環境では使用不可

# 文の埋め込みベクトルを生成する関数
def sen2vec(sentence):
    # Convert the given sentence to a vector representation using a text embedding model.  
    input_body = {"inputText": sentence}
    try:
        response = bedrock_client.invoke_model(
            body=json.dumps(input_body),
            modelId=modelId,
            accept="application/json",
            contentType="application/json",
            )
        response_body = json.loads(response.get("body").read())
        vector = np.array(response_body.get("embedding"))
        return vector
    except Exception as e:
        print(e)

サンプルファイル documents-jp.txt に、5つの文章が保存されているので、それぞれをベクトル化します。

In [15]:
# ドキュメントのファイルを読み込み用の配列を初期化
with open("documents-jp.txt") as doc:
    num_records = len(doc.readlines())

doc_array=np.empty(shape=(num_records), dtype="S255")
embed_array = np.zeros(shape=(num_records, 1536)) # v2 では 1536 → 1024

# ドキュメントのファイルを読み込み、ベクトル化データを配列へ格納
with open("documents-jp.txt") as doc:
    for num, line in enumerate(doc, 0):
      doc = line.strip('\n')
      doc_array[num] = doc.encode("UTF-8")
      embed_array[num] = sen2vec(doc)

print("ベクトル化データの配列のサイズ")
print(embed_array.shape)
print("----")
print("ベクトル化データの配列の内容") 
print(embed_array)
print("----")

ベクトル化データの配列のサイズ
(5, 1536)
----
ベクトル化データの配列の内容
[[ 1.41406250e+00  3.80859375e-02  5.82031250e-01 ...  7.66601562e-02
  -9.49218750e-01 -1.75781250e-01]
 [ 1.03906250e+00 -2.83203125e-01  5.74218750e-01 ... -4.71191406e-02
  -5.23437500e-01  2.29492188e-01]
 [ 1.05468750e+00 -1.19781494e-03  6.75781250e-01 ... -4.33349609e-03
  -4.45312500e-01  2.04101562e-01]
 [ 1.37500000e+00  6.95312500e-01 -5.24902344e-02 ... -1.04980469e-01
  -7.65625000e-01  2.24609375e-02]
 [ 8.43750000e-01  3.43750000e-01  1.99218750e-01 ...  1.70898438e-01
   1.58203125e-01  1.17675781e-01]]
----


類似性検索を使用して類似度を算出します。  
行列ベクトルストアから関連ドキュメントを引き出すために類似性検索を使用するインタラクティブなクエリを実行  
類似度のしきい値として0.5を使用して、幻影(ハルシネーション)を限定します。  
ドキュメントストアのコース名に関連するクエリは、成功した結果につながります。  
結果のいずれも0.5のしきい値を超えない場合、「わかりません」が応答になります。

In [16]:
my_text = "AWSの研修"
#my_text = "AnyCompany社の勤務に関する情報"
#my_text = "車のタイヤの交換"

def search_doc(text):
    embed_query = sen2vec(text)
    denominator = norm(embed_array, axis=1) * norm(embed_query)
    similarity = embed_array.dot(embed_query) / denominator
    max_value = max(similarity)
    max_value_index = similarity.argmax()
    print("\nテキスト: " + text)
    print("\n類似度")
    print(similarity)
    print("\n")
    if max_value > 0.5:
        print("類似性が高いドキュメント:\n" + doc_array[max_value_index].decode('UTF-8'))
    else:
        print("どのドキュメントも類似性が 50% 未満です。")
    # print("\nSimilarity vector used for document selection")
    # return max_value_index

search_doc(my_text)



テキスト: AWSの研修

類似度
[0.85657059 0.73833652 0.61924214 0.37798074 0.32037387]


類似性が高いドキュメント:
AWSのトレーニングでArchitecting on AWSは3日間のコースです。


類似度の高いテキストが検索できました。
別の文章をインプットして類似度が高い文章を探してみましょう。

In [ ]:
my_text=""

search_doc(my_text)